In [1]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

import nltk
import pandas as pd
import numpy as np

In [34]:
import joblib

In [2]:
reviews = pd.read_csv("data.csv")

In [3]:
reviews.shape

(9987, 2)

In [4]:
reviews.head()

,review,overall
0,Don't like it. It will not hold my cigarettes....,1.0
1,love Four Stars,4.0
2,Wonderful item. I have purchased several of th...,4.0
3,I just love it! I think this is my six or seve...,5.0
4,made well but will only hold a small pack of c...,4.0


In [5]:
indetifier = pd.read_csv("identifier.csv")

In [6]:
indetifier.shape

(9987, 2)

### creating data set of reviews and labels (removed now)

In [8]:
data_set = []
for index,row in reviews.iterrows():
    data_set.append(row['review'])

In [31]:
print(len(data_set))

9987


In [9]:
data_set[:2]

["Don't like it. It will not hold my cigarettes. Not long enough. One Star",
 'love Four Stars']

In [10]:
# initializaing stemmer

stemmer = PorterStemmer()

In [11]:
stoplist = stopwords.words('english') + list(punctuation)

In [12]:
import re

In [63]:
def preprocess(doc, sent=False):
#     print(doc)
    # lower case
    # tokenize into words
    words = word_tokenize(doc)
    # remove stop words
    words = [re.sub("[\'./-=+].+", "" ,word.lower()) for word in words if word not in stoplist]
    
    
    words = [stemmer.stem(word) for word in words if len(word) > 2]
    
    document = " ".join(words)
#     print("document",document)
    if sent:
        return document
    else:
        return words

In [64]:
# preprocess review data with word set in return

message_set = [preprocess(message) for message in data_set]

In [68]:
# preprocess review data with sent set in return

sent_set = [preprocess(message, sent=True) for message in data_set]

In [15]:
len(message_set)

9987

In [65]:
message_set[:2]

[['like', 'hold', 'cigarett', 'not', 'long', 'enough', 'one', 'star'],
 ['love', 'four', 'star']]

In [69]:
sent_set[:3]

['like hold cigarett not long enough one star',
 'love four star',
 'wonder item purchas sever they last forev unless lose happi purchas buxton made last made last']

### SENTENCE EMBEDDING

In [44]:
# from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [32]:
# Building a TF IDF matrix out of the corpus of reviews
# from sklearn.feature_extraction.text import TfidfVectorizer
# td = TfidfVectorizer(max_features = 4500)
# X = td.fit_transform(message_set).toarray()

In [35]:
# joblib.dump(td.vocabulary_, 'tfidf_20220102.pkl')

['tfidf_20220102.pkl']

In [46]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [52]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(message_set)]

In [53]:
tagged_data[:2]

[TaggedDocument(words=['like', 'hold', 'cigarett', 'not', 'long', 'enough', 'one', 'star'], tags=[0]),
 TaggedDocument(words=['love', 'four', 'star'], tags=[1])]

In [54]:
model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1, epochs = 100)

# model.wv.vocab contains model vocabulary

'\nvector_size = Dimensionality of the feature vectors.\nwindow = The maximum distance between the current and predicted word within a sentence.\nmin_count = Ignores all words with total frequency lower than this.\nalpha = The initial learning rate.\n'

In [55]:
# saving sent embedding model

model.save("doc_embedding_20220102")

In [57]:
import gensim

### Trying loading the model and checking if it  works as expected

In [58]:
model_loaded = gensim.models.Word2Vec.load("doc_embedding_20220102")

In [61]:
model_loaded.infer_vector(["i am happy"])

array([-0.01171344, -0.01917688, -0.02077239, -0.00681774,  0.01457975,
       -0.0174458 ,  0.02329998, -0.00936267,  0.00748427,  0.00060759,
       -0.01225773,  0.01710643, -0.02454663, -0.02379553,  0.01906421,
        0.00457406, -0.00600684,  0.00250608,  0.02475584,  0.01102187],
      dtype=float32)

In [73]:
len(sent_set)

9987

In [122]:
# vectorizing all sentences

X = []
for sentence in message_set:
    X.append(model.infer_vector(sentence))

In [123]:
X = np.array(X)

In [124]:
X[0].shape

(20,)

In [121]:
X[:5]

array([[-2.8934722 ,  0.8105569 , -0.05500418, -0.6458565 ,  0.23412736,
         1.336904  , -0.4396123 ,  1.3420455 , -0.11662147, -0.5038103 ,
         0.95538855, -0.24892607,  0.06631743, -0.01830056,  1.1267383 ,
         0.6176976 ,  1.5180448 , -0.20286061, -0.72187585,  0.01762974],
       [-0.17614418,  0.18999436,  0.12506874,  0.02836764, -0.18944907,
         0.25617853, -0.11302615,  0.86333686, -0.68892837,  0.1860896 ,
        -0.5057833 , -0.5399364 ,  0.46944556, -0.05551057,  0.20048162,
         0.30074483,  0.82069665, -0.25850135, -1.0541384 , -0.1693759 ],
       [-1.091864  ,  2.6368256 ,  0.963515  ,  0.58820194, -1.2787174 ,
         2.4591606 ,  0.45860162,  0.09472281, -1.0917852 , -0.92316073,
        -2.7503798 , -0.9171574 , -0.25222987, -2.0764556 , -0.26186475,
         1.5073183 ,  0.45111078,  0.5243225 , -1.0061351 , -0.01246951],
       [-1.0858907 ,  1.3540184 , -1.8365468 ,  1.9294231 , -0.36378846,
         0.4840808 ,  2.609053  ,  1.1450601 ,  

In [116]:
y = reviews.overall.values

In [117]:
# Splitting into training & test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,
                                                    random_state = 0)

In [118]:
X_test.shape

(2997, 20)

## MODEL TRAINING

In [104]:
from sklearn.linear_model import LogisticRegression

In [105]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [106]:
clf.score(X_test, y_test)

0.6329662996329662

In [107]:
clf.predict(X_test)[:100]

array([1., 5., 5., 3., 3., 2., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 3.,
       4., 5., 5., 5., 4., 5., 4., 5., 5., 5., 4., 5., 4., 1., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 3., 5., 5., 5., 5., 5., 4.,
       5., 5., 5., 3., 5., 5., 5., 5., 5., 1., 5., 5., 5., 5., 5., 1., 5.,
       5., 5., 5., 5., 5., 5., 5., 4., 5., 3., 5., 5., 5., 5., 5., 5., 5.,
       5., 3., 5., 4., 5., 5., 4., 5., 5., 5., 4., 5., 5., 5., 2.])

In [108]:
# saving the model

joblib.dump(clf, 'model20220102.pkl', compress=1)

['model20220102.pkl']

### Loading the model again to check if it works as expected

In [109]:
model_clf = joblib.load('model20220102.pkl')

In [112]:
model_clf.predict(X_test[1:2]).tolist()[0]

5.0

In [114]:
X_test[1:2].shape

(1, 20)